In [14]:
from bs4 import BeautifulSoup
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

In [15]:
mainPaths =[ 
    './templates/contents/contents_1/귀어귀촌개념.html', 
    './templates/contents/contents_1/귀어귀촌준비절차.html', 
    './templates/contents/contents_1/어업면허허가.html', 
    './templates/contents/contents_1/어업정의.html', 
    './templates/contents/contents_2/귀어정책지원교육.html', 
    './templates/contents/contents_2/양식장및어선임대사업.html', 
    './templates/contents/contents_2/어촌이사혜택.html', 
    './templates/contents/contents_2/청년어촌정착지원금.html', 
    './templates/contents/contents_5/어업종류.html', 
    './templates/contents/contents_5/낚시및해루질_금어기.html', 
    './templates/contents/contents_5/낚시및해루질_낚시금지구역.html', 
    './templates/contents/contents_5/낚시및해루질_종류별낚시설명및초보자팁.html', 
    './templates/contents/contents_5/낚시및해루질_지역별낚시명소.html', 
    './templates/contents/contents_5/낚시및해루질_지역별낚시용품점.html'
    ]


# mainPath4 = './templates/contents/contents_5/어업종류.html'


# mainPath5 = './templates/contents/contents_3/이상생태현상_굴폐사.html'
# mainPath6 = './templates/contents/contents_3/이상생태현상_산호초백화.html'
# mainPath7 = './templates/contents/contents_4/IoTAI활용스마트양식기술소개.html'

file_path = mainPaths
contentList = []

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,     # 총 길이
    chunk_overlap=50,   # 겹치는 길이
    separators=["\n\n", "\n", ".", " ", ""],  # 끊는 우선순위
)



for i in file_path:
    with open(i, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    folder = os.path.basename(os.path.dirname(i)) 
    page = os.path.splitext(os.path.basename(i))[0]  
    soup = BeautifulSoup(html_content, 'html.parser')
    sections = soup.select('[name="section"]')
    titles = None
    contents = None
    title = None
    content = None
    for j in sections:
        titles = j.select('[name="content_title"]')
        contents = j.select('[name="content_text"]')
        # 둘 다 1개 이상일 때
        if len(titles) > 1 and len(contents) > 1:
            count = min(len(titles), len(contents))  # 길이 맞추기
            for k in range(count):
                title = titles[k].get_text(strip=True)

                content = contents[k].get_text(strip=True)
                

                chunks = splitter.split_text(content)
                for k, chunk in enumerate(chunks):
                    contentList.append([title, chunk, folder, page])

        # 하나만 있을 경우 (정확히 하나일 때만 출력)
        elif len(titles) == 1 and len(contents) == 1:
            title = titles[0].get_text(strip=True)
            content = contents[0].get_text(strip=True)
            chunks = splitter.split_text(content)
            for k, chunk in enumerate(chunks):
                contentList.append([title, chunk, folder, page])
        else:
            print(f"{f}의 제목 또는 내용이 없습니다.")
            # print(j)

df = pd.DataFrame(contentList,columns=["title", "content", "folder", "page"])
df



,title,content,folder,page
0,귀어/귀어업,어업활동을 하기 위해 타지에서부터 어촌으로 이사와 거주하는 것,contents_1,귀어귀촌개념
1,귀촌/귀어촌,귀촌/귀어촌어업활동과 관련없이 타지에서 어촌으로 이주하는 것,contents_1,귀어귀촌개념
2,귀어귀촌 준비절차,"STEP 01.귀어를 준비하는 사람은 수산 관련기관 홈페이지에서 정보를 수집하고, ...",contents_1,귀어귀촌준비절차
3,귀어귀촌 준비절차,".행정 절차를 완료한 후, 수립한 계획에 따라 사업을 본격적으로 추진한다.STEP ...",contents_1,귀어귀촌준비절차
4,어업면허,어업면허는 특정한 수면을 독점적으로 사용할 수 있도록 국가가 권리를 부여하는 제도입...,contents_1,어업면허허가
...,...,...,...,...
237,4. 전라도-남해·서남해,"갯바위낚시 명소. 다금바리, 돌돔 잡이 가능.3. 성산포항동부권 대표 선상낚시 출항...",contents_5,낚시및해루질_지역별낚시명소
238,5. 경상도·부산,"1. 통영 욕지도 / 매물도선상낚시 명소. 참돔, 부시리, 갈치 다양.갯바위 포인트...",contents_5,낚시및해루질_지역별낚시명소
239,6. 제주도,"1. 제주시 한림항 / 조천항제주 북부 항구 방파제 낚시.자리돔, 벵에돔, 감성돔 ...",contents_5,낚시및해루질_지역별낚시명소
240,낚시용품점,낚시를 하려면 먼저 가장 중요한 것은 낚시에 필요한 장비들을 준비하는 것입니다.낚시...,contents_5,낚시및해루질_지역별낚시용품점


In [16]:
# 2. 문서 객체 생성
documents = [
    Document(page_content=row['content'], metadata={"title": row['title'], "folder": row['folder'], "page": row['page']})
    for _, row in df.iterrows()
]

embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

vectorstore = FAISS.from_documents(documents=documents, embedding=embedding_model)

vectorstore.save_local("chatbot/vectore_store/contents")